In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import re
from datetime import datetime 
from dateutil import parser

import pandas as pd
import numpy as np
from collections import defaultdict


In [2]:
fighter_dat = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_train_data.csv')
fighter_names_date = fighter_dat[['red_fighter','blue_fighter', 'event_date']]
df_sorted = fighter_names_date.sort_values(by=['red_fighter', 'event_date'], ascending=[True, False])


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from dateutil import parser
from collections import defaultdict

def fighter_odds_search(red_fighter, blue_fighters, event_dates, driver):

    search_name = red_fighter
    search_box = driver.find_element(By.ID, "search-box1")
    search_box.clear()
    search_box.send_keys(search_name)
    search_box.send_keys(Keys.RETURN)

    try: 
        table = driver.find_element(By.CSS_SELECTOR, "table.content-list")
    except:
        print(f"Found ONE matches: {red_fighter}")
        odds_dic = scrape_odds(driver, blue_fighters, event_dates)
        print(f'Confirming return vals: {odds_dic}')
        driver.back()  # Go back to the search results page
        return odds_dic

    rows = table.find_elements(By.TAG_NAME, "tr")
    possible_matches = []

    for row in rows:
        fighter_link = row.find_element(By.CSS_SELECTOR, "td a")
        fighter_name = fighter_link.text.lower()

        # Check if all parts of the search name are in the fighter's name
        if all(term in clean_string(fighter_name) for term in clean_string(search_name)) or \
            all(term in clean_string(search_name) for term in clean_string(fighter_name)):
            possible_matches.append(fighter_link)

    if possible_matches:
        print(f"Found possible matches: {[link.text for link in possible_matches]}")
        odds_dic = defaultdict(list)
        for link in possible_matches:
            link.click()
            return_dic = scrape_odds(driver, blue_fighters, event_dates)
            
            for key, value in return_dic.items():
                if len(value)!=0:
                    odds_dic[key].extend(value)
                    
            driver.back()  # Go back to the search results page
        if len(odds_dic['red_fighter']) != 0:
            odds_dic['og_red_fighter'].extend([red_fighter] * len(odds_dic['red_fighter']))
        return odds_dic
            # time.sleep(2)  # Allow time for the page to reload
    else:
        print(f"No match found for '{search_name}'.")


def scrape_odds(driver, blue_fighters, dates):

    red_fighter_row = driver.find_elements(By.CSS_SELECTOR, "tr.main-row") #red fighter row 
    odds_dic = {'blue_fighter':[], 'open_blue':[], 'close1_blue':[], 'close2_blue':[],
            'red_fighter':[], 'open_red':[], 'close1_red':[], 'close2_red':[], 'event_date':[],
            'og_blue_name':[]}

    for i in range(0, len(red_fighter_row)):

        red_name = red_fighter_row[i].find_element(By.CSS_SELECTOR, "th.oppcell").text
        blue_row = red_fighter_row[i].find_element(By.XPATH, "following-sibling::tr[1]")
        blue_name = blue_row.find_element(By.CSS_SELECTOR, "th.oppcell a").text
        # blue_name = clean_string_simple(blue_name)
        match, og_blue_name = is_two_way_partial_match(blue_name, blue_fighters)
        try:
            event_date_elements = blue_row.find_elements(By.CSS_SELECTOR, "td.item-non-mobile")
            current_event_date = event_date_elements[0].text.strip()
            dt = parser.parse(current_event_date).date()
            dt = str(dt)
        except:
            dt = None
        
        if match == 'True': 
            #case when page values give n/a, see Allan Nascimento
            try:
                
                moneylines_blue = blue_row.find_elements(By.CSS_SELECTOR, "td.moneyline span")
                open_range_blue = moneylines_blue[0].text
                closing_range_blue = [moneylines_blue[1].text, moneylines_blue[2].text]

                moneylines_red = red_fighter_row[i].find_elements(By.CSS_SELECTOR, "td.moneyline span")
                open_range_red = moneylines_red[0].text
                closing_range_red = [moneylines_red[1].text, moneylines_red[2].text]

                odds_dic['red_fighter'].append(red_name)
                odds_dic['blue_fighter'].append(blue_name)

                odds_dic['open_blue'].append(open_range_blue)
                odds_dic['close1_blue'].append(closing_range_blue[0])
                odds_dic['close2_blue'].append(closing_range_blue[1])

                odds_dic['open_red'].append(open_range_red)
                odds_dic['close1_red'].append(closing_range_red[0])
                odds_dic['close2_red'].append(closing_range_red[1])

                odds_dic['event_date'].append(dt)
                odds_dic['og_blue_name'].append(og_blue_name)
            except:
                continue

    return odds_dic

def clean_string(s):
    return s.lower().replace(" ", " ").replace("-", " ").replace(".", "").replace("'","")

def is_two_way_partial_match(scraped_name, search_names):
    scraped_parts = clean_string(scraped_name)
    for name in search_names:
        search_parts = clean_string(name)
        if all(part in scraped_parts for part in search_parts) or \
           all(part in search_parts for part in scraped_parts):
            return 'True', name  # Match found
    return 'False', 'None'  # No match found

import random 
import time
import undetected_chromedriver as uc

def get_fighter_odds(fighter_df):
    
    url = 'https://www.bestfightodds.com/'
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64)...",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)...",
        # add more realistic user agents
    ]

    # options = Options()
    # options.add_argument(f'user-agent={random.choice(user_agents)}')
    # driver = webdriver.Chrome(options=options)

    options = uc.ChromeOptions()
    # options.add_argument('--headless=new')
    options.add_argument(f'user-agent={random.choice(user_agents)}')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--profile-directory=Default")
    options.add_argument("--incognito")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.get(url)
    print(driver.current_url)  # see where you landed
    print(driver.title)   
    red_groups = fighter_df.groupby('red_fighter')

    odds_df = pd.DataFrame()
    for red_name, group in red_groups:
        print(red_name)
        blue_fighters = group['blue_fighter'].values
        # blue_fighters = [clean_string_simple(name) for name in blue_fighters]
        event_dates = group['event_date'].values

        time.sleep(random.uniform(1,2))

        odds_dic = fighter_odds_search(red_name, blue_fighters, event_dates, driver)
        row = pd.DataFrame(odds_dic)
        odds_df = pd.concat([odds_df, row], ignore_index=True)

    driver.quit()

    return odds_df

In [ ]:
# df_sub = df_sorted[df_sorted['blue_fighter']=='Benoit Saint Denis']
test = df_sorted[
    (df_sorted['red_fighter'] == 'Dustin Poirier') |
    (df_sorted['blue_fighter'] == 'Dustin Poirier')
][['red_fighter', 'blue_fighter', 'event_date']]

odds_df = get_fighter_odds(test)


In [ ]:
df_sorted[
    (df_sorted['red_fighter'] == 'Zhang Weili') |
    (df_sorted['blue_fighter'] == 'Zhang Weili')
][['red_fighter', 'blue_fighter']].head(15)

In [143]:
df_sorted[df_sorted['red_fighter']== 'Abubakar Nurmagomedov']['blue_fighter'].head()

4822    Elizeu Zaleski dos Santos
4589          Gadzhi Omargadzhiev
Name: blue_fighter, dtype: object

'Elizeu Zaleski', 'Gadzhi Omargadzhiev', 'Ustarmagomed Gadzhidaudov', 'Ustarmagomed Gadzhidaudov'

In [13]:
odds_df = get_fighter_odds(df_sorted)


https://www.bestfightodds.com/
UFC & MMA Odds & Betting Lines | Best Fight Odds
AJ Dobson
Found possible matches: ['A.J. Dobson', 'Aj Dobson']
AJ Fletcher
Found possible matches: ['A.J. Fletcher', 'Aj Fletcher', 'Jaime Fletcher', 'Jeff Fletcher']
Aaron Phillips


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.158)
Stacktrace:
	GetHandleVerifier [0x0xb41af3+62339]
	GetHandleVerifier [0x0xb41b34+62404]
	(No symbol) [0x0x982123]
	(No symbol) [0x0x971b30]
	(No symbol) [0x0x98f8f2]
	(No symbol) [0x0x9f5cbc]
	(No symbol) [0x0xa10089]
	(No symbol) [0x0x9ef1b6]
	(No symbol) [0x0x9be7a2]
	(No symbol) [0x0x9bf644]
	GetHandleVerifier [0x0xdb6683+2637587]
	GetHandleVerifier [0x0xdb1a8a+2618138]
	GetHandleVerifier [0x0xb6856a+220666]
	GetHandleVerifier [0x0xb58998+156200]
	GetHandleVerifier [0x0xb5f12d+182717]
	GetHandleVerifier [0x0xb49a38+94920]
	GetHandleVerifier [0x0xb49bc2+95314]
	GetHandleVerifier [0x0xb34d0a+9626]
	BaseThreadInitThunk [0x0x76245d49+25]
	RtlInitializeExceptionChain [0x0x776bd1ab+107]
	RtlGetAppContainerNamedObjectPath [0x0x776bd131+561]


In [6]:
odds_df.head()

,blue_fighter,open_blue,close1_blue,close2_blue,red_fighter,open_red,close1_red,close2_red,event_date,og_blue_name,og_red_fighter
0,Elizeu Zaleski,-110,-140,+110,Abubakar Nurmagomedov,-110,-133,+120,2023-06-03,Elizeu Zaleski dos Santos,Abubakar Nurmagomedov
1,Gadzhi Omargadzhiev,+155,+115,+140,Abubakar Nurmagomedov,-180,-175,-135,2022-10-22,Gadzhi Omargadzhiev,Abubakar Nurmagomedov
2,Ustarmagomed Gadzhidaudov,+300,+250,+250,Abubakar Vagaev,-400,-325,-325,2022-03-27,Gadzhi Omargadzhiev,Abubakar Nurmagomedov
3,Ustarmagomed Gadzhidaudov,-105,+250,+275,Abubakar Vagaev,-125,-400,-325,2021-10-04,Gadzhi Omargadzhiev,Abubakar Nurmagomedov


In [4]:
#50 min runtime 
odds_df = get_fighter_odds(df_sorted)
odds_df.to_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\odds_df.csv')

Found possible matches: ['A.J. Dobson', 'Aj Dobson']
Found possible matches: ['A.J. Fletcher', 'Aj Fletcher', 'Jaime Fletcher', 'Jeff Fletcher']
Found possible matches: ['Aaron Phillips', 'Brandon Phillips']
Found possible matches: ['Aaron Riley', 'Jason Riley']
Found possible matches: ['Aaron Simpson', 'Tessa Rose Simpson']
Found possible matches: ['Abdul Razak Alhassan', 'Razak Al-Hassan', 'Razak Al-Hussan']
Found possible matches: ['Abel Trujillo', 'Albert Trujillo', 'Alberto Trujillo', 'Robert Trujillo']
Found possible matches: ['Abu Azaitar', 'Ottman Azaitar*lightweight Bout']
Found possible matches: ['Abubakar Nurmagomedov', 'Khabib Nurmagomedov', 'Omar Nurmagomedov', 'Umar Nurmagomedov', 'Abubakar Adamu', 'Abubakar Vagaev']
Found possible matches: ['Abus Magomedov', 'Abusupiyan Magomedov', 'Hasbulla Magomedov', 'Magomed Magomedov']
Found possible matches: ['Adam Milstead']
Found possible matches: ['Adam Wieczorek', 'Adam Amado']
Found possible matches: ['Adrian Yanez', 'Desiree 

In [ ]:
'Diego Lopes'

In [141]:
odds_df[odds_df['red_fighter']=='Cub Swanson']

,blue_fighter,open_blue,close1_blue,close2_blue,red_fighter,open_red,close1_red,close2_red,event_date
1082,Billy Quarantillo,-150,-210,-163,Cub Swanson,+110,+130,+175,2024-12-15
1083,Andre Fili,-155,-250,-225,Cub Swanson,+130,+175,+200,2024-06-30
1084,Hakeem Dawodu,-250,-320,-225,Cub Swanson,+200,+175,+250,2023-08-12
1085,Jonathan Martinez,-205,-225,-205,Cub Swanson,+175,+165,+190,2022-10-15
1086,Darren Elkins,+135,+130,+155,Cub Swanson,-155,-190,-160,2021-12-18
1087,Daniel Pineda,-110,-164,-152,Cub Swanson,-110,+132,+137,2020-12-12
1088,Kron Gracie,-265,-189,-175,Cub Swanson,+185,+150,+162,2019-10-12
1089,Shane Burgos,+135,-185,-172,Cub Swanson,-175,+140,+160,2019-05-04
1090,Renato Moicano,-165,-310,-300,Cub Swanson,+125,+250,+260,2018-08-04
1091,Brian Ortega,+110,-120,-113,Cub Swanson,-150,-107,+100,2017-12-09


In [137]:
odds_df.shape

(5036, 9)